In [3]:
# Load required packages
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import zipfile
import nltk
import os
import string

In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [3]:
file_dir = 'C:\\Users\\yashd\\Desktop\\URAP\\Code\\Topic_text_csv\\'
output_dir = 'C:\\Users\\yashd\\Desktop\\URAP\\Code\\'
topic_files = os.listdir(file_dir)

In [4]:
topics_df = pd.DataFrame({'text_id' : np.array(list(map(lambda x: x[:-4], topic_files)))})

In [5]:
# initializing empty arrays for each topic
os.chdir(file_dir)
info_array = np.array([])
motivation_array = np.array([])
neutral_array = np.array([])
strategy_array = np.array([])


for i in range(0, 504):
    interview_df = pd.read_csv(topic_files[i])
    interview = interview_df['content']
    # Checks if an interview is skipped and accounts for it
    unique, counts = np.unique(interview_df['topic'], return_counts=True)
    if (len(counts) < 4):
        diff = {'information', 'motivation', 'neutral', 'strategy'} - set(unique)
        while (diff != set()):
            skipped = diff.pop()
            if skipped == 'information':
                info_array = np.append(info_array, 0)
            elif skipped == 'motivation':         
                motivation_array = np.append(motivation_array, 0)
            elif skipped == 'neutral':
                neutral_array = np.append(neutral_array, 0)
            elif skipped == 'strategy':
                strategy_array = np.append(strategy_array, 0)
    
    for topic, count in zip(unique, counts):
        if topic == 'information':
            info_array = np.append(info_array, count)
        elif topic == 'motivation':         
            motivation_array = np.append(motivation_array, count)
        elif topic == 'neutral':
            neutral_array = np.append(neutral_array, count)
        elif topic == 'strategy':
            strategy_array = np.append(strategy_array, count)

In [45]:
topics_df

,text_id,Information,Strategy,Motivation,Neutral,similarity_to_job
0,00-11-59-17,15.000,4.000,2.000,13.000,0.810
1,00-48-76-89,9.000,10.000,1.000,13.000,0.774
2,00-60-29-25,10.000,8.000,3.000,15.000,0.889
3,00-64-40-74,30.000,28.000,3.000,11.000,0.803
4,00-67-53-14,5.000,8.000,3.000,19.000,0.359
...,...,...,...,...,...,...
499,98-65-25-06,30.000,26.000,11.000,16.000,0.792
500,99-33-66-89,37.000,23.000,9.000,42.000,0.821
501,99-41-75-39,16.000,18.000,6.000,36.000,0.895
502,99-50-93-22,25.000,17.000,11.000,36.000,0.780


In [8]:
content_df = pd.read_csv('C:\\Users\\yashd\\Desktop\\URAP\\Code\\merged_content.csv')
topic_and_content_df = pd.concat([content_df, topics_df], axis=1, join="inner")

In [9]:
# Takes in TXT and JOB and measures the similarity of the text to the JOB
def similarity_to_job(txt, job):
    job_classifier = classifier(txt, job.lower(), multi_label = True)
    return job_classifier['scores'][0]

In [11]:
# Dictionary with key-value pairs representing the job category data before and after processing
job_categories = ['Construction/Building/Bricklaying', 'Motor-mechanics', 'Plumbing', 'Secretarial', 'Tailoring',
 'Teacher/Early Childhood Development', 'Catering/Food service (cooking, serving, restaurants)', 'Machining & fitting', 'Welding',
 'Hairdressing/Barber shop', 'Electrical work (wiring/installation/phone-TV repair)', 'Agriculture/Farming/Animal Rearing/Fishing',
 'Accounting', 'Secretary/Accountant/Business Management', 'Carpentry']
job_dict = dict(zip(topic_and_content_df['trainee_training_area'].unique().tolist(), job_categories))

In [15]:
%%time
# Creates and appends job similarity scores to array
jobs = np.array([])
for i in range(len(topics_df)):
    jobs = np.append(jobs, similarity_to_job(topic_and_content_df['compiled_content'][i], job_dict[topic_and_content_df['trainee_training_area'][i]]))

Wall time: 1h 24min 52s


In [66]:
topics_df['Information'] = info_array
topics_df['Strategy'] = strategy_array
topics_df['Motivation'] = motivation_array
topics_df['Neutral'] = neutral_array
topics_df['similarity_to_job'] = jobs

topics_df['Information'] = topics_df['Information'].apply(int)
topics_df['Strategy'] = topics_df['Strategy'].apply(int)
topics_df['Motivation'] = topics_df['Motivation'].apply(int)
topics_df['Neutral'] = topics_df['Neutral'].apply(int)
topics_df['similarity_to_job'] = topics_df['similarity_to_job'].apply(lambda x: np.round(x, 3))

In [67]:
topics_df

,text_id,Information,Strategy,Motivation,Neutral,similarity_to_job
0,00-11-59-17,15,4,2,13,0.810
1,00-48-76-89,9,10,1,13,0.774
2,00-60-29-25,10,8,3,15,0.889
3,00-64-40-74,30,28,3,11,0.803
4,00-67-53-14,5,8,3,19,0.359
...,...,...,...,...,...,...
499,98-65-25-06,30,26,11,16,0.792
500,99-33-66-89,37,23,9,42,0.821
501,99-41-75-39,16,18,6,36,0.895
502,99-50-93-22,25,17,11,36,0.780


In [70]:
topics_df.to_csv('C:\\Users\\yashd\\Desktop\\URAP\\Code\\topics.csv')